In [1]:
import os
import json
import requests
import base64

import numpy as np
import pandas as pd


In [2]:
base_dir =os.path.expanduser('~')
home_dir =f'{base_dir}/jpx'


home_dir

'/home/jupyter/jpx'

In [3]:
import sys
import datetime

sys.path.append(home_dir)
sys.path.append(home_dir + '/predict/src')
from predictor import ScoringService
#from fundamental.predictor import ScoringServiceFundamental



import importlib
#import api
#importlib.reload(api.fetch_api)
from api.fetch_api import (
    fetch_jquant_data,
    call_refresh_api,
    call_jquants_api,
    match_type_df
)
from api import token

refreshtoken = token.refreshtoken
idtk = call_refresh_api(refreshtoken)['idToken']

paramdict = {}
paramdict["datefrom"] = "2020-03-27"
paramdict["dateto"] = "2021-05-07"
paramdict["includedetails"] = "True"


In [4]:
DATASET_DIR=f'{home_dir}/data'
MODEL_PATH=f'{home_dir}/predict/model'


inputs = ScoringService.get_inputs(DATASET_DIR)


load_data = ['stock_list', 'stock_price', 'stock_fin', 'stock_labels', 'nikkei_article','tdnet' ]

dfs = ScoringService.get_dataset(inputs, load_data)
codes = dfs['stock_list']['Local Code'].unique() #全銘柄

INFO : 2021-05-08 07:53:53,775 : read stock_list
INFO : 2021-05-08 07:53:53,797 : read stock_price
INFO : 2021-05-08 07:54:04,081 : read stock_fin
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
INFO : 2021-05-08 07:54:04,664 : read tdnet
INFO : 2021-05-08 07:54:06,968 : read nikkei_article
INFO : 2021-05-08 07:54:09,113 : read stock_labels


In [10]:
# df_news = pd.read_csv(f"{DATASET_DIR}/nikkei_article.csv.gz")
# df_news

In [11]:
dfs.keys()

dict_keys(['stock_list', 'stock_price', 'stock_fin', 'tdnet', 'nikkei_article', 'stock_labels'])

In [13]:
from tqdm import tqdm
df_stock_lists_list = []
for code in tqdm(codes):
    lists = pd.DataFrame(call_jquants_api(paramdict, idtk, "lists", f"{code}")['list'])
    df_stock_lists_list.append(lists)

100%|██████████| 3711/3711 [06:28<00:00,  9.54it/s]


In [14]:
stock_list = pd.concat(df_stock_lists_list)#.drop_duplicates()
#stock_list = stock_list[dfs['stock_list'].columns].reset_index(drop=True)

stock_list = match_type_df(stock_list, dfs['stock_list'])


In [15]:
stock_list_drop_col = stock_list.drop(['Effective Date', 'IssuedShareEquityQuote ModifyDate'], axis=1)


stock_list_api_list = []
for i in stock_list['Local Code'].unique():
    df_local = stock_list[stock_list['Local Code']==i]
    df_local_recent = df_local.sort_values(by='Effective Date', ascending=False).iloc[[0]]
    stock_list_api_list.append(df_local_recent)
    
stock_list_api = pd.concat(stock_list_api_list).reset_index(drop=True)


In [18]:
# stock_list更新分保存
#stock_list_api.to_csv('stock_list.csv.gz', compression='gzip', index=False)

In [5]:
#%%capture output

df_stock_price_api = fetch_jquant_data(codes=codes, paramdict=paramdict, target="prices", idtk=idtk)


100%|██████████| 3711/3711 [53:01<00:00,  1.17it/s]


In [6]:

df_stock_price_api = match_type_df(df_stock_price_api, dfs['stock_price'])

In [15]:
df_stock_price_api

,Local Code,EndOfDayQuote Date,EndOfDayQuote Open,EndOfDayQuote High,EndOfDayQuote Low,EndOfDayQuote Close,EndOfDayQuote ExchangeOfficialClose,EndOfDayQuote Volume,EndOfDayQuote CumulativeAdjustmentFactor,EndOfDayQuote PreviousClose,EndOfDayQuote PreviousCloseDate,EndOfDayQuote PreviousExchangeOfficialClose,EndOfDayQuote PreviousExchangeOfficialCloseDate,EndOfDayQuote ChangeFromPreviousClose,EndOfDayQuote PercentChangeFromPreviousClose,EndOfDayQuote VWAP
0,1301,2020/03/27,2718.0,2757.0,2687.0,2757.0,2757.0,139800.0,1.0,2689.0,2020/03/26,2689.0,2020/03/26,68.0,2.529,2719.655
1,1301,2020/03/30,2600.0,2624.0,2507.0,2619.0,2619.0,98400.0,1.0,2757.0,2020/03/27,2757.0,2020/03/27,-138.0,-5.005,2576.886
2,1301,2020/03/31,2600.0,2605.0,2510.0,2545.0,2545.0,46700.0,1.0,2619.0,2020/03/30,2619.0,2020/03/30,-74.0,-2.826,2550.565
3,1301,2020/04/01,2517.0,2557.0,2498.0,2517.0,2517.0,56000.0,1.0,2545.0,2020/03/31,2545.0,2020/03/31,-28.0,-1.100,2522.818
4,1301,2020/04/02,2467.0,2539.0,2446.0,2512.0,2512.0,44400.0,1.0,2517.0,2020/04/01,2517.0,2020/04/01,-5.0,-0.199,2503.723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,9997,2021/04/27,1206.0,1207.0,1181.0,1186.0,1186.0,207500.0,1.0,1219.0,2021/04/26,1219.0,2021/04/26,-33.0,-2.707,1189.085
267,9997,2021/04/28,1178.0,1178.0,1154.0,1168.0,1168.0,305000.0,1.0,1186.0,2021/04/27,1186.0,2021/04/27,-18.0,-1.518,1166.699
268,9997,2021/04/30,1165.0,1173.0,1157.0,1159.0,1159.0,251300.0,1.0,1168.0,2021/04/28,1168.0,2021/04/28,-9.0,-0.771,1163.398
269,9997,2021/05/06,1150.0,1189.0,1148.0,1169.0,1169.0,406400.0,1.0,1159.0,2021/04/30,1159.0,2021/04/30,10.0,0.863,1171.180


In [7]:
df_stock_price_api.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 995319 entries, 0 to 270
Data columns (total 16 columns):
 #   Column                                           Non-Null Count   Dtype  
---  ------                                           --------------   -----  
 0   Local Code                                       995319 non-null  int64  
 1   EndOfDayQuote Date                               995319 non-null  object 
 2   EndOfDayQuote Open                               995319 non-null  float64
 3   EndOfDayQuote High                               995319 non-null  float64
 4   EndOfDayQuote Low                                995319 non-null  float64
 5   EndOfDayQuote Close                              995319 non-null  float64
 6   EndOfDayQuote ExchangeOfficialClose              995319 non-null  float64
 7   EndOfDayQuote Volume                             995319 non-null  float64
 8   EndOfDayQuote CumulativeAdjustmentFactor         995319 non-null  float64
 9   EndOfDayQuote Prev

In [9]:
stock_price_concat = pd.concat([dfs['stock_price'].reset_index(drop=True), df_stock_price_api])
stock_price_concat.to_csv('stock_price.csv.gz', compression='gzip', index=False)

In [20]:
stock_price_concat.drop_duplicates().to_csv('stock_price.csv.gz', compression='gzip', index=False)  # 5390207

In [17]:
aaa = stock_price_concat[stock_price_concat.duplicated()]

In [19]:
aaa['EndOfDayQuote Date'].unique()

array(['2020/03/27', '2020/03/30', '2020/03/31', '2020/04/01',
       '2020/04/02', '2020/04/03', '2020/04/06', '2020/04/07',
       '2020/04/08', '2020/04/09', '2020/04/10', '2020/04/13',
       '2020/04/14', '2020/04/15', '2020/04/16', '2020/04/17',
       '2020/04/20', '2020/04/21', '2020/04/22', '2020/04/23',
       '2020/04/24', '2020/04/27', '2020/04/28', '2020/04/30',
       '2020/05/01', '2020/05/07', '2020/05/08', '2020/05/11',
       '2020/05/12', '2020/05/13', '2020/05/14', '2020/05/15',
       '2020/05/18', '2020/05/19', '2020/05/20', '2020/05/21',
       '2020/05/22', '2020/05/25', '2020/05/26', '2020/05/27',
       '2020/05/28', '2020/05/29', '2020/06/01', '2020/06/02',
       '2020/06/03', '2020/06/04', '2020/06/05', '2020/06/08',
       '2020/06/09', '2020/06/10', '2020/06/11', '2020/06/12',
       '2020/06/15', '2020/06/16', '2020/06/17', '2020/06/18',
       '2020/06/19', '2020/06/22', '2020/06/23', '2020/06/24',
       '2020/06/25', '2020/06/26', '2020/06/29', '2020/

In [32]:


stockfins_from_api = fetch_jquant_data(codes=codes, paramdict=paramdict, target="stockfins", idtk=idtk)





100%|██████████| 3523/3523 [10:05<00:00,  5.82it/s]


In [83]:
stockfins_from_api = match_type_df(stockfins_from_api, dfs['stock_fin'])

In [85]:

stock_stockfins_concat = pd.concat([dfs['stock_fin'].loc[:'2020-01-01'].reset_index(drop=True), stockfins_from_api])
#stock_stockfins_concat.to_csv('stock_fin.csv.gz', compression='gzip', index=False)

In [89]:


stocklabels_from_api = fetch_jquant_data(codes=codes, paramdict=paramdict, target="stocklabels", idtk=idtk)



100%|██████████| 3523/3523 [39:06<00:00,  1.50it/s] 


In [94]:
stocklabels_from_api = match_type_df(stocklabels_from_api, dfs['stock_labels'])
stocklabels_from_api_concat = pd.concat([dfs['stock_labels'].loc[:'2020-01-01'].reset_index(drop=True), stocklabels_from_api])
stocklabels_from_api_concat.to_csv('stock_labels.csv.gz', compression='gzip', index=False)

In [92]:
stocklabels_from_api_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4394888 entries, 0 to 299
Data columns (total 11 columns):
 #   Column         Dtype  
---  ------         -----  
 0   base_date      object 
 1   Local Code     int64  
 2   label_date_5   object 
 3   label_high_5   float64
 4   label_low_5    float64
 5   label_date_10  object 
 6   label_high_10  float64
 7   label_low_10   float64
 8   label_date_20  object 
 9   label_high_20  float64
 10  label_low_20   float64
dtypes: float64(6), int64(1), object(4)
memory usage: 402.4+ MB
